In [1]:
import cv2
import pytesseract
from matplotlib import pyplot as plt
import numpy as np

# Set the path to the Tesseract executable (change it based on your installation)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
# Read the input image
image = cv2.imread("../Allen.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imshow("GrayScale Image", gray)
cv2.imshow("Original Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [76]:
# #Noise removal function
# def noise_removal(image):
#     kernel = np.ones((1, 1), np.uint8)
#     image = cv2.dilate(image, kernel, iterations=1)
#     kernel = np.ones((1, 1), np.uint8)
#     image = cv2.erode(image, kernel, iterations=1)
#     image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
#     image = cv2.medianBlur(image, 3)
#     return (image)

# no_noise = noise_removal(gray)
# # cv2.imwrite("preprocess/no_noise.jpg", no_noise)

# plt.imshow(gray,'gray',vmin=0,vmax=255)
# cv2.imshow('Noise Reduction', no_noise)
# cv2.imshow("GrayScale", gray)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [10]:
# Apply thresholding to get a binary image
_, binary_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
B1 = cv2.adaptiveThreshold(gray, maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 5, C = 15)

# Apply morphological operations to remove noise and enhance text regions
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
# processed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
processed_image = cv2.morphologyEx(kernel, cv2.MORPH_CLOSE, kernel)

cv2.imshow("Thresh Image", binary_image)
cv2.imshow("Adaptive Thresh", B1)
cv2.imshow("Processed image", processed_image)
# cv2.imshow("Adaptive Thresh", B1)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [62]:
# # Find contours in the processed image
# contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Iterate through each contour and extract text using Tesseract
# for contour in contours:
#     x, y, w, h = cv2.boundingRect(contour)

#     # Filter out small contours to remove noise
#     if cv2.contourArea(contour) > 100:
#         # Extract the region of interest (ROI) containing the text
#         roi = image[y:y + h, x:x + w]

#         # Use Tesseract OCR to extract text from the ROI
#         text = pytesseract.image_to_string(roi, config='--psm 6')  # Adjust psm (page segmentation mode) as needed

#         # Print the extracted text
#         print("Extracted Text:\n", text)

#         # Draw a bounding box around the detected text
#         cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

In [78]:
#bounding boxes for each text and words
# Iterate through each contour and extract text using Tesseract
contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)

    # Filter out small contours to remove noise
    if cv2.contourArea(contour) > 100:
        # Extract the region of interest (ROI) containing the text
        roi = image[y:y + h, x:x + w]
        # Use Tesseract OCR to extract text from the ROI
        text = pytesseract.image_to_string(roi, config='--psm 6')  # Adjust psm (page segmentation mode) as needed
        
        # Print the extracted text
        print("Extracted Text:\n", text)
        
        # Use Tesseract OCR to get individual words or characters
        custom_config = r'--oem 3 --psm 6 outputbase digits'
        results = pytesseract.image_to_boxes(roi, config=custom_config).splitlines()

        # Draw bounding boxes for each word or character
        for line in results:
            word_data = line.split()
            x_start, y_start, x_end, y_end = map(int, word_data[1:5])
            
            # Draw a bounding box around the word or character
            cv2.rectangle(image, (x + x_start, y + y_start), (x + x_end, y + y_end), (0, 255, 0), 2)


Extracted Text:
 POU ale suUCH a WONGCTIUL SUT,
A gift to keep, a gift to care for
and a gift to love now and forever.
You are like the heaven's gift.
The one I have been hoping for
and dreaming about my entire life.



In [79]:
# Display the image with bounding boxes
cv2.imshow('Text Extraction', image)
cv2.waitKey(0)
cv2.destroyAllWindows()